# Imports

In [8]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
import numpy.matlib
import pandas as pd
import time
import os
from os import listdir
from os.path import join, isfile

!pip install jupyter-dash
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import requests
import io

In [4]:
path = r'C:\Users\bobby\Documents\Github\PD-Watch-Dashboard\Data\data\110001'

# Data Processing

In [9]:
url = 'https://github.com/BobbyWilt/PD-Watch-Dashboard/blob/main/Data/data/110001/110001_trials.npy'
url2 = 'https://github.com/BobbyWilt/PD-Watch-Dashboard/raw/main/Data/data/110001/110001_trials.npy'
response = requests.get(url)
# response.raise_for_status()
data = np.load(io.BytesIO(response.content))

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [12]:
data = np.load(join(path, '110001' + '_trials.npy')).astype(np.float64)

In [35]:
# import csv
# np.savetxt("data.csv", data, delimiter=",")
arr = data
# arrReshaped = arr.reshape(arr.shape[0], -1)
for i in range(data.shape[0]):
    df_temp = pd.DataFrame(data[i])
    df_temp.to_csv(f'data_segment_{i+1}.csv')
# arrReshaped = data[0]
# test = pd.DataFrame(arrReshaped)
# saving reshaped array to file.
# np.savetxt('data.csv', arrReshaped)
# test.to_csv('test' + '.csv')

# Figures

# Dashboard

In [ ]:
# Build App
df = px.data.tips()
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )# Run app and display result inline in the notebook
app.run_server(mode='external')